In [1]:
import os
from time import sleep
from pandas import DataFrame
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.common.exceptions import StaleElementReferenceException
from selenium.webdriver.common.by import By

In [2]:

chrome_options = Options()
chrome_options.add_argument("--no-sandbox")  # Bypass OS security model
chrome_options.add_argument("--user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/95.0.4638.54 Safari/537.36")
#chrome_options.add_argument("--headless") # uncomment this to use chrome in headless mode
chrome_options.add_argument("window-size=1024,768")  # Open Browser in maximized mode
chrome_options.add_argument("disable-infobars")  # Disabling infobars
chrome_options.add_argument("--disable-extensions")  # Disabling extensions
chrome_options.add_argument("--disable-dev-shm-usage")  # Overcome limited resource problems
chrome_options.add_experimental_option("prefs", {'download.default_directory': os.environ['DOWNLOAD_LOCATION']})

In [7]:
import unacademy_scraper.ua_consts as ua_consts

In [23]:
# initiate a headless chrome
driver = webdriver.Chrome(options=chrome_options, executable_path= os.environ['CHROME_DRIVER'])

In [24]:
driver.get(ua_consts.UA_BASE_URL)

In [26]:
login_button = driver.find_element_by_css_selector('.e13239452')
login_button.click()

In [28]:
# Locate the input field using the placeholder attribute
mobile_input_field = driver.find_element_by_css_selector('input[placeholder="Enter your mobile number"]')

# Take mobile number from user
mobile_number = input("Enter Mobile Number : ")

# Insert the mobile number
mobile_input_field.send_keys(mobile_number)

In [29]:
request_otp_button = driver.find_element_by_css_selector('.ehkw11q4.aquilla-button.button.css-1dik306-SendOtp')
request_otp_button.click()

In [32]:
otp = input("Enter OTP : ")

In [33]:
# Locate all OTP input fields using CSS selector
otp_fields = driver.find_elements_by_css_selector('.aquilla-otpinput-inputfield.otpinput-inputfield.css-d9udvv')

# Populate each field with a digit from the OTP
for i, digit in enumerate(otp):
    otp_fields[i].send_keys(digit)

IndexError: list index out of range

In [22]:
driver.get(ua_consts.COURSE_LINK_INTRUMENTATION_AND_PROCESS_CONTROL)